## NVD CVE JSON Mirror Maintenance

### This is a collection of routines to maintain a mirror of the NVD JSON CVE repository.

To use:
    <ul>
    <li>install mongodb
    <li>create a database called nvd
    <li>create collections: maincol, modscol, metacol
    <li>identify a working and a config directory - working dir is to download JSON files , config dir is where you put
        the .mongo.env file
    </ul>
        

In [435]:
#author: Ray Zupancic - Grumblesoft.com
import requests
from pathlib import Path
import json
from pymongo import MongoClient
from bson import json_util
from bson.objectid import ObjectId
import pandas as pd
import glob
import sys
import zipfile
import datetime
import pathlib

class MongodbConn():
    
    def __init__(self,configpath='', configname=".mongo.env"):
        if configpath == '':
            #set a home directory path where all JSON files and mongodb config live
            self.configpath = str(Path.home())+"/" + configname
        else:
            self.configpath = configpath
        
        try:
            #read the params    
            self.par = self._get_params(self.configpath)    
            self.cli = self._connect_mongo(self.par)
            #validate that the db exists
            db_names = self.cli.list_database_names()
            db = self.par["database"]
            if not db in db_names:
                raise Exception("db: {} not found on instance".format(db))
            self.mydb = self.cli[db]
            #connect to or create the collections
            self.maincol = self.mydb[self.par["maincollection"]]
            self.modscol = self.mydb[self.par["modscollection"]]
            self.metacol = self.mydb[self.par["metacollection"]]
            self.intecol = self.mydb[self.par["intecollection"]]
        except ValueError as v:
            print(repr(v))
        except Exception as e:
            print("Could not connect to the collection")
            print(repr(e))
            sys.exit()

        
    def _get_params(self,filename):
        ''' Public method to read values from a file and get parameters
            place your mongodb connect information in this file
            server,<yourservername or IP>
            username,<username>
            password,<mongopassword>
            authSource,<db you are authenticating against>
            database,<your-database>
            collection,<your-collection>
        '''
        myparams = {}
        with open(filename) as myfile:
            for line in myfile:
                key, value = line.partition("=")[::2]
                myparams[key.strip()] = value.strip()
        return myparams

    
    def _connect_mongo(self,params):
        '''
            Public method to connect to a mongo instance using params from input
            params - a dictionary with user, password, server and database
        '''
        client = MongoClient(params.get("server"),\
                              username=params.get("username"),\
                              password=params.get("password"),\
                              authSource=params.get("authSource"),\
                              authMechanism='SCRAM-SHA-1')
        return client

    
class QueryMongo(MongodbConn):
    '''
        queryMongo class is used to apply find functions against a mongodb 
        inherits: mongoConn - connection instance
    '''
    
    def __init__(self,configpath='', configname=".mongo.env"):
        '''
            Intializer method - instance the super class initializer
        '''
        super().__init__(configpath, configname)
    
    def get_count(self, col):
        '''
            Public method to return the document count for a given collection
            input: collection
            output: integer 
        '''
        return col.count_documents({})
    
    
    def get_fields(self,data_l,col):
        '''
            Public method to query mongodb
            input: list of fields []
            output: cursor mongodb
        '''
        qs = '"_id":0'
        for item in data_l:
            qs += ',"'+item+'":1'
        print(qs)
        cursor = col.find({},{qs})
        return cursor

    
    def get_fields(self,filter_d,proj_d, col):
        '''
            Public method to query a field 
            input: filter (dictionary), projection (dictionary), collection
            output: cursor (mongodb)
        '''
        cursor = col.find(filter_d,proj_d)
        return cursor
    
    
    def get_cve(self,cve, col):
        '''
            Public method to query a collection for a given CVE
            input: cve (string), collection
            output: dictionary
        '''
        try:
            cve_d = col.find_one({"CVE_ID":cve})
            if not bool(cve_d):
                raise ValueError("CVE: {} does not exist".format(cve))
        except Exception as e:
            print(repr(e))
        return cve_d

    
    def get_cve_last_modified(self,CVE, col):
        '''
            Public method to get last modified date
            input: string (CVE ID), collection
            output: cursor
        '''
        filter_d = {"CVE_ID":{ "$regex": CVE }}
        proj_d = {"_id":0,"lastModifiedDate":1}
        cr = col.find(filter_d, proj_d)
        return cr
    
    
    def get_true_mods_CVEID(self):
        '''
            Public method to get the mod CVEs that are not new. This uses pandas to do the intersection
            input: na
            output: DataFrame
        '''
        filt = {}
        proj = {'CVE_ID':1,'_id':0}
        cr1 = self.maincol.find(filt, proj)
        df1 = pd.DataFrame(list(cr1))
        proj = {'CVE_ID':1,'_id':0,'lastModifiedDate':1}
        cr2 = self.modscol.find(filt, proj)
        df2 = pd.DataFrame(list(cr2))
                       
        return df2.merge(df1,how='inner',on='CVE_ID')
    
    def get_new_cves_from_mods(self):
        '''
            Public method to get the mod CVEs that are new. This uses pandas to do the intersection
            input: na
            output: DataFrame
        '''
        filt = {}
        proj = {'CVE_ID':1,'_id':0}
        cr1 = self.maincol.find(filt, proj)
        df1 = pd.DataFrame(list(cr1))
        proj = {'CVE_ID':1,'_id':0,'lastModifiedDate':1}
        cr2 = self.modscol.find(filt, proj)
        df2 = pd.DataFrame(list(cr2))
        #use the indicator to get the left outer join with exclusion
        df =  pd.merge(df2,df1,on='CVE_ID',how='outer',indicator=True)        
        return df[df['_merge']=='left_only']
    
    
    def get_CVEID_newer_than(self,datestring, col="maincol"):
        '''
            Public method to return a DataFrame of all CVEs and LastModifiedDAte newer than the string 
            supplied. Supply a date in for format 2015-09-15 (yyyy-mm-dd). The caller can optionally supply a 
            collection, the default is maincol.This uses pandas to do the intersection
            input: datestring (str), collection
            output: DataFrame
        '''
        QueryMongo.valid_date(datestring)
        try:
            filt = {}
            proj = {'CVE_ID':1,'_id':0,'lastModifiedDate':1}
            #get cursor
            cr = self.modscol.find(filt, proj)
            #convert cursor to DataFrame
            df = pd.DataFrame(list(cr))
            df["Updated"] = pd.to_datetime(df["lastModifiedDate"])
            df['Updated'] = pd.to_datetime(df['Updated']).apply(lambda x: x.replace(tzinfo=None)) 
            ts = pd.Timestamp(datestring)
            #return results greater than the timestamp provided
            df=df[df['Updated']>ts]
        except Exception as e:
            print(repr(e))
            
        return df[df["Updated"]>ts]
    
    def _parse_json(self, data):
        '''
            Private method to handle BSON
            input: BSON
            output: JSON
        '''
        return json.loads(json_util.dumps(data))
    
    @staticmethod
    def valid_date(date_text):
        try:
            datetime.datetime.strptime(date_text, '%Y-%m-%d')
        except ValueError:
            raise ValueError("Incorrect data format, should be YYYY-MM-DD")
            

class UpdateMongo(QueryMongo):
    
    
    def __init__(self, workingpath = '', configpath='', configname=".mongo.env"):
        '''
            Intializer method - instance the super class initializer
            input: string (working directory), string (config directory), string (ini file)
        '''
        super().__init__(configpath, configname)
        self.workingpath = workingpath
        
        
    def get_NVD_files(self,file_l):
        ''' Public method to download NVD datafiles in zip format, the URL is:
            https://nvd.nist.gov/feeds/json/cve/1.1/nvdcve-1.1-[year].json.gz
            inputs: d_path - directory path as string
        '''
        for file in file_l:
            try:
                #need the URL, which for NVD files is an embedding of the year
                url = 'https://nvd.nist.gov/feeds/json/cve/1.1/' +file
                f = requests.get(url, allow_redirects=True)
                #save the file
                f_path = self.workingpath + file
                open(f_path, 'wb').write(f.content)
                f.close()
            except Exception as e:
                print("error on download - check NVD JSON files availability")
                print(repr(e))
            try:
                #unzip the NVD files
                with zipfile.ZipFile(f_path, 'r') as zip_ref:
                    zip_ref.extractall(self.workingpath)
            except Exception as e:
                print("error on unzipping files")
                print(repr(e))

    def rebuild_yearly_NVD_files(self,download_files=False):
        '''
            Public method to rebuild the yearly data from NVD. This will empty the main cve collection and
            rebuild it with new NVD files. Generally, one would want to use the modified file from NVD to add
            modifications and insertions rather than rebuilding frequently 
            input: boolean (download NVD files from NVD?)
            output: list (inserted files)
        '''
        json_files_l = []
        #get the present year - we will get all files ffrom 2002 to present year
        year = datetime.date.today().year
        years = range(2002,year+1)
        for year in years:
            json_files_l.append('nvdcve-1.1-' + str(year) + '.json.zip')
        #download the files and unzip
        if download_files:
            self.get_NVD_files(json_files_l)
        #delete all docs in main collection
        self.remove_all_cves(self.maincol)
        
        json_files_l = [item[:-4] for item in json_files_l]
        self.manage_bulk_insert(json_files_l)
        
        return json_files_l
        
                

    def insert_json_file(self, filepath):
        '''
            insert any json file into collection
        '''

        with open(filepath) as jsonf :
            file_data = json.load(jsonf)
            if isinstance(file_data, list):
                self.mycol.insert_many(file_data)
            else:
                self.mycol.insert_one(file_data)

        client.close()


    def _insert_flattened_json_file(self, file, col):
        '''
            public function to flatten and insert a yearly NVD JSON file into a collection of CVEs
            input: file name - NVD yearly JSON file
            output: InsertManyResult object
        '''
        df = pd.DataFrame()
        with open(file) as data_file:
           data = json.load(data_file)
        #normalize out the CVEs
        df = pd.json_normalize(data,'CVE_Items')
        df.rename({'cve.CVE_data_meta.ID':'CVE_ID'}, axis=1,inplace=True)
        #change the awkward index name
        df.rename_axis('CVE')
        #MongoDB doesn't like '.' in key names
        df.columns = df.columns.str.replace(".", "_")
        #insert records into MongoDB
        result =col.insert_many(df.to_dict('records'))


    def manage_bulk_insert(self,filelist):
        '''
            Public method to allow for loading all NVD JSON files into a collection
        '''
        try:
            for file in filelist:
                try:
                    filecheck = pathlib.Path(self.workingpath + file)
                    if not filecheck.exists():
                        raise FileNotFoundError("file: not found on path".format(filecheck))
                except Exception as e:
                    print(repr(e))
                file = self.workingpath + file
                print("inserting file: {}".format(file))
                self._insert_flattened_json_file(file,self.maincol)
        except Exception as e:
            print(repr(e))
            
    def reload_NVD_mod_file(self,download=True):
        '''
            Public method to download and insert the latest NVD CVE Modifications file
            into the modscol collection. Optionally the file in the working directory 
            can be loaded without being downloaded (must exist on working dir). This doesn't apply the mods
            to the main collection (maincol)
            input: boolean (download flag)
            output: integer (records inserted)
        '''
        if download:
            self.get_NVD_files(["nvdcve-1.1-modified.json.zip"])
        #empty the mod collection
        self.remove_all_cves(self.modscol)
        file = self.workingpath+"nvdcve-1.1-modified.json"
        self._insert_flattened_json_file(file,self.modscol)
    
    def _get_mods_list_cursor(self):
        '''
            Private method to query for CVE mods and return a cursor of CVEs that have changed
            input: na
            output: cursor
        '''
        try:
            filter_d = {}
            proj_d = {}
            cr = self.modscol.find(filter_d)
            if self.modscol.count_documents({}) == 0:
                raise ValueError ("no modcves found")
                cr = None
        except Exception as e:
            print(repr(e))
        return cr
    
    def _get_list_cursor(self, col):
        '''
            Private method to query a collection and return a cursor 
            input: collection
            output: cursor
        '''
        try:
            filter_d = {}
            proj_d = {'CVE_ID':1}
            cr = col.find(filter_d)
            if col.count_documents({}) == 0:
                raise ValueError ("no cves found")
        except Exception as e:
            print(repr(e))
            cr = None
        return cr
    
    def get_cve_exists(self,cve, col):
        '''
            Public method to query a collection for a given CVE and return exists versus not exists as bool
            input: cve (string), collection
            output: boolean
        '''
        
        return bool(col.find_one({"CVE_ID":cve}))
    
    def update_main_from_mods(self):
        '''
            Public method to remove CVEs modified and add CVE modified and new CVEs into the main cve collection. The method
            outputs the total number of changes as a list of mods and additions
            input: na
            output: [mods (integer), additions (integer)]
        '''
        mods_l = []
        for cve in self._get_mods_list_cursor():
            cve_id = cve.get("CVE_ID")
            #check if exists
            if self.get_cve_exists(cve_id, self.maincol):
                #if exists, add it to a list to be modified
                mods_l.append(cve.get("_id"))
            
        #delete the mods
        #r_mods = self.maincol.delete_many({"_id":{'$in':mods_l}})
        #add all the mods and additions
        #r_adds =self.maincol.insert_many(list(self._get_mods_list_cursor()))
        return mods_l
    
    def update_main_from_mods_dataframe(self):
        '''
            Public method to remove CVEs modified and add CVE modified and new CVEs into the main cve collection. The method
            outputs the total number of changes as a list of mods and additions
            input: na
            output: [mods (integer), additions (integer)]
        '''
        
        df1 = pd.DataFrame(list(self._get_list_cursor(self.modscol)))
        df2 = pd.DataFrame(list(self._get_list_cursor(self.maincol)))
        df = pd.merge(df1,df2,on="CVE_ID", how = "inner")
        
        #save as a list of CVE#s
        cve_l = df["CVE_ID"].to_list()
        #delete the true mods from main
        result1=um.maincol.delete_many({"CVE_ID": {"$in":cve_l}})
        #insert the mods
        cur = self._get_list_cursor(self.modscol)
        result2=um.maincol.insert_many(df1.to_dict('records'))
        
        return result1,result2
        
    
    def remove_cve(self,cve,col):
        '''
            Public method to remove a single record from mongodb and return the deleted count
            input: list of fields []
            output: integer 
        '''
        try:
            query = { "CVE_ID":cve }
            result = col.delete_one(query)
            if result.deleted_count != 1:
                raise ValueError("CVE: {} not deleted".format(cve))
        except Exception as e:
            print(repr(e))
        return result.deleted_count
    
    def remove_all_cves(self,col):
        '''
            Public method to remove all records from mongodb collection and return the deleted count
            input: list of fields []
            output: integer 
        '''
        try:
            query = {}
            result = col.delete_many(query)
        except Exception as e:
            print(repr(e))
        return result.deleted_count



#### instantiate query and update instances

In [436]:
um = UpdateMongo("/home/rzupancic/CVE/")

#### check the count of CVEs for a given string as a test
Example: {"CVE_ID":{ "$regex": "^CVE-2014"}} will count the total CVEs that begin with CVE-2014
Example: 2014 has 8845 (this could change)

In [100]:
filter_d = {"CVE_ID":{ "$regex": "^CVE-2014"}}
proj_d = {"_id":0,"CVE_ID":1}
cr = um.get_fields(filter_d, proj_d,um.maincol)
len(list(cr))

8845

#### There are two types of files we are interested in: yearly and mods

In [ ]:
years = list(range(2002,2021))
mod = ["modified"]

#### Re-insert (or initially insert) the yearly files after optionally downloading the latest batch from NVD
The boolean indicates whether to download. If false, simply re-insert the files in the <br>
working directory (these must exist if download is False)

In [460]:
um.rebuild_yearly_NVD_files(False)

inserting file: /home/rzupancic/CVE/nvdcve-1.1-2002.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2003.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2004.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2005.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2006.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2007.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2008.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2009.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2010.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2011.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2012.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2013.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2014.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2015.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2016.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2017.json
inserting file: /home/rzupancic/CVE/nvdcve-1.1-2018.json
inserting file: /home/rzupancic

['nvdcve-1.1-2002.json',
 'nvdcve-1.1-2003.json',
 'nvdcve-1.1-2004.json',
 'nvdcve-1.1-2005.json',
 'nvdcve-1.1-2006.json',
 'nvdcve-1.1-2007.json',
 'nvdcve-1.1-2008.json',
 'nvdcve-1.1-2009.json',
 'nvdcve-1.1-2010.json',
 'nvdcve-1.1-2011.json',
 'nvdcve-1.1-2012.json',
 'nvdcve-1.1-2013.json',
 'nvdcve-1.1-2014.json',
 'nvdcve-1.1-2015.json',
 'nvdcve-1.1-2016.json',
 'nvdcve-1.1-2017.json',
 'nvdcve-1.1-2018.json',
 'nvdcve-1.1-2019.json',
 'nvdcve-1.1-2020.json']

#### Reload the NVD Modifications file - this emptys the cvemods collection and rebuilds it. 
use reload_NVD_mods_file(False) to use an existing mods file in the working dir.<BR>
reload_NVD_mods_file(True) will download the latest mods file and then insert <br>
 it into the modscol collection (True is the default)

In [389]:
um.reload_NVD_mod_file(True)

#### get counts of the two NVD collections 

In [445]:
print("Total CVEs: {}".format(um.get_count(um.maincol)))
print("Total CVE Mods: {}".format(um.get_count(um.modscol)))

Total CVEs: 153866
Total CVE Mods: 901


#### Get the last modified date of a CVE by CVEID from a given collection

In [245]:
d1 = um.get_cve_last_modified("CVE-2005-0366", um.modscol)
print(list(d1))

[{'lastModifiedDate': '2020-12-10T15:17Z'}]


In [244]:
d1 = um.get_cve_last_modified("CVE-2005-0366", um.maincol)
print(list(d1))

[{'lastModifiedDate': '2008-09-10T19:35Z'}]


#### Get all info from a CVE and prettify the output

In [257]:
cve = um.get_cve("CVE-2005-0366", um.maincol)
print(json.dumps(um._parse_json(cve),indent=4))

{
    "_id": {
        "$oid": "5fd6a2fc18f709e5be049550"
    },
    "publishedDate": "2005-05-02T04:00Z",
    "lastModifiedDate": "2008-09-10T19:35Z",
    "cve_data_type": "CVE",
    "cve_data_format": "MITRE",
    "cve_data_version": "4.0",
    "CVE_ID": "CVE-2005-0366",
    "cve_CVE_data_meta_ASSIGNER": "cve@mitre.org",
    "cve_problemtype_problemtype_data": [
        {
            "description": [
                {
                    "lang": "en",
                    "value": "NVD-CWE-Other"
                }
            ]
        }
    ],
    "cve_references_reference_data": [
        {
            "url": "http://eprint.iacr.org/2005/033",
            "name": "http://eprint.iacr.org/2005/033",
            "refsource": "MISC",
            "tags": []
        },
        {
            "url": "http://eprint.iacr.org/2005/033.pdf",
            "name": "http://eprint.iacr.org/2005/033.pdf",
            "refsource": "MISC",
            "tags": []
        },
        {
            "url": 

#### Remove a CVE by CVEID from a given collection

In [ ]:
um.remove_cve("CVE-2020-15999",um.cvecol)

#### Get a dataframe of true mods - CVEs that actually existed and were modified

In [443]:
df = um.get_true_mods_CVEID()
print(df[['CVE_ID','lastModifiedDate']])  
print("Length of DataFrame: {}".format(df.shape))

            CVE_ID   lastModifiedDate
0    CVE-2007-0720  2020-12-15T02:02Z
1    CVE-2007-1320  2020-12-15T23:52Z
2    CVE-2007-1321  2020-12-15T23:52Z
3    CVE-2007-1322  2020-12-15T23:49Z
4    CVE-2007-1366  2020-12-15T23:49Z
..             ...                ...
200  CVE-2020-9488  2020-12-16T06:15Z
201  CVE-2020-9876  2020-12-15T20:15Z
202  CVE-2020-9883  2020-12-15T20:15Z
203  CVE-2020-9941  2020-12-15T20:15Z
204  CVE-2020-9952  2020-12-18T19:10Z

[205 rows x 2 columns]
Length of DataFrame: (205, 2)


#### Get a dataframe of the new, not in main,  CVEs that were in the Mods file

In [394]:
df = um.get_new_cves_from_mods()
print(df[['CVE_ID','lastModifiedDate']])  
print("Length of DataFrame: {}".format(df.shape))

             CVE_ID   lastModifiedDate
35   CVE-2018-16243  2020-12-17T18:05Z
52   CVE-2019-14476  2020-12-18T15:41Z
53   CVE-2019-14477  2020-12-17T20:31Z
54   CVE-2019-14478  2020-12-17T16:32Z
55   CVE-2019-14479  2020-12-17T20:12Z
..              ...                ...
896   CVE-2020-9988  2020-12-15T20:15Z
897   CVE-2020-9989  2020-12-15T20:15Z
898   CVE-2020-9991  2020-12-15T23:15Z
899   CVE-2020-9996  2020-12-15T20:15Z
900   CVE-2020-9999  2020-12-16T05:15Z

[696 rows x 2 columns]
Length of DataFrame: (696, 3)


In [253]:
df[df.loc[:,"CVE_ID"]=="CVE-2005-0366"]

,lastModifiedDate,CVE_ID
16,2020-12-10T15:17Z,CVE-2005-0366


#### get a dataframe of CVEs newer than a given date

In [243]:
df = um.get_CVEID_newer_than("2020-12-10", "modcol")
print(df)

      lastModifiedDate          CVE_ID             Updated
16   2020-12-10T15:17Z   CVE-2005-0366 2020-12-10 15:17:00
32   2020-12-10T00:15Z   CVE-2015-5184 2020-12-10 00:15:00
38   2020-12-10T23:15Z  CVE-2016-15001 2020-12-10 23:15:00
45   2020-12-10T20:59Z  CVE-2017-11422 2020-12-10 20:59:00
54   2020-12-10T20:59Z   CVE-2017-3222 2020-12-10 20:59:00
..                 ...             ...                 ...
810  2020-12-11T00:15Z   CVE-2020-8694 2020-12-11 00:15:00
811  2020-12-11T05:15Z   CVE-2020-8908 2020-12-11 05:15:00
812  2020-12-10T12:54Z   CVE-2020-8919 2020-12-10 12:54:00
813  2020-12-10T12:54Z   CVE-2020-8920 2020-12-10 12:54:00
818  2020-12-11T13:13Z   CVE-2020-9301 2020-12-11 13:13:00

[405 rows x 3 columns]


#### check if a CVE_ID exists in a given collection

In [259]:
um.get_cve_exists("CVE-2005-0366",um.maincol)

True

#### Create a small function to return a list of mods that exist in the main collection
Note that this will be crazy slow


In [402]:
def get_list(um):
    mods_l = []
    for cve in um._get_mods_list_cursor():
        #print(cve)
        cve_id = cve.get("CVE_ID")
        #check if exists in main collection
        if um.get_cve_exists(cve_id, um.maincol):
            #if exists, add it to a list to be modified
            mods_l.append(cve.get("_id"))
    return mods_l

%timeit t_mods_l = get_list(um)
print(len(t_mods_l))


1min 28s ± 2.05 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
205


#### Try the same thing using a DataFrame

In [447]:
%timeit df2 = um.get_true_mods_CVEID()
df2.shape

609 ms ± 3.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


(205, 2)

#### how to delete many on a list
<ul>
    <li>get a dataframe that contains the _id's that you want to delete</li>
<li>make sure the _id column didn't get renamed (as in a merge): <i>df.rename(columns={"_xyz?":"_id"},inplace=True)</i></li>
    <li>get a list in the form: <b>ObjectId('5fd537df18f709e5be046598'),</b> :<i> id_l =df.to_list()</i></li>
    <li>run a delete many against the filter: um.maincol.delete_many( {"_id": {"in":id_l}}  )</li>
<li> execute the delete:<br> db.collection.delete_many({"_id": {"$in":id_l}})  </li>
</ul>

In [438]:
#use df2 dataframe from previous cell
cve_l = df2["CVE_ID"].to_list()

In [439]:
#delete all of the true mods from the main collection
result=um.maincol.delete_many({"CVE_ID": {"$in":cve_l}})

In [440]:
#should match with the number of true mods 
result.deleted_count

0

#### insert all mods into main collection

In [465]:
main_cves_count = um.get_count(um.maincol)
print("Total CVEs in main collection: {}".format(main_cves_count))
mod_df=um.get_true_mods_CVEID()
new_df=um.get_new_cves_from_mods()
mod_cves_count = mod_df.shape[0]
new_cves_count = new_df.shape[0]
print("True Mods count: {}".format(mod_cves_count))
print("Total new CVEs in mods file: {}".format(new_cves_count))
print("Expect to see: {} in main collection when mods file applied".format(main_cves_count +new_cves_count))

Total CVEs in main collection: 153866
True Mods count: 205
Total new CVEs in mods file: 696
Expect to see: 154562 in main collection when mods file applied


In [466]:
#update the main collection by applying the mods file
r1,r2 = um.update_main_from_mods_dataframe()

In [467]:
print("Deleted Count: {}".format(r1.deleted_count))
print("Inserted Count: {}".format(len(r2.inserted_ids)))
print("Total CVEs in main collection: {}".format(um.get_count(um.maincol)))

Deleted Count: 205
Inserted Count: 901
Total CVEs in main collection: 154562


In [455]:
type(r2)

pymongo.results.InsertManyResult

In [458]:
len(r2.inserted_ids)

901